# Hosting a Boltz Serving App

[Boltz](https://github.com/jwohlwend/boltz) is an open-source biomolecular structure prediction model with performance on par with Alphafold3. This notebook details a simple boltz prediction task as well as how to serve a prediction endpoint using FastAPI and Union Serving.

## Overview
- Define a [remote object](https://docs.union.ai/byoc/user-guide/development-cycle/union-remote/) to interact with the Union cluster
- Materialize an Artifact representing the [model on Huggingface](https://huggingface.co/boltz-community/boltz-1)
- Create an [ImageSpec](https://docs.union.ai/byoc/user-guide/development-cycle/image-spec#imagespec) definition for use throughout
- Create a simple prediction workflow using [Actors](https://docs.union.ai/byoc/user-guide/core-concepts/actors/#actors)
- Define a FastAPI serving endpoint
- Deploy the app via Union [Serving](https://docs.union.ai/byoc/user-guide/core-concepts/serving/#serving)

## Setup
- Install the `union` package
- Create a config file via `union create login` and make it available at the environment variable below

### UnionRemote

The following cell will refer to your config file and create a UnionRemote object that's used throughout the rest of the notebook. This object allows you to register entities, trigger executions, and retrieve outputs in a programmatic way.

In [ ]:
import os
import subprocess
from union import UnionRemote, ImageSpec, ActorEnvironment, FlyteFile, FlyteDirectory, workflow, Resources, Artifact
from flytekit.configuration import Config

os.environ["UNION_CONFIG"] = "~/.union/config_serving.yaml"

remote = UnionRemote(config=Config.auto(config_file=os.path.expanduser(os.environ["UNION_CONFIG"])))

### Cache Model from Huggingface

UnionRemote has a convenience function for caching models from Huggingface as Union Artifacts. You'll need to create an API token on HF and then upload it via `union create secret --name HF_TOKEN`. You'll also have to create an admin key via `union create api-key admin --name UNION_API_KEY`.

This will run a workflow that fetches the model and emits an Artifact. You can view the execution in the console, as well as the model in the Artifacts tab once the workflow has completed.

In [ ]:
from union.remote import HuggingFaceModelInfo
info = HuggingFaceModelInfo(repo="boltz-community/boltz-1")

cache_exec = remote._create_model_from_hf(
    info=info, 
    hf_token_key="HF_TOKEN", 
    union_api_key="UNION_API_KEY",
)

cache_exec = cache_exec.wait(poll_interval=2)
cache_exec.outputs

### Create an ImageSpec Definition

ImageSpec is an easy and flexible way of defining the images you'll be using throughout your workflow and in your apps. A number of options are built in for PyPI packages, conda packages, etc. 

We define a number of PyPI packages as well as the `build-essential` APT bundle for Boltz. Finally, we install Boltz via an arbitrary RUN command.

Of note here is the use of the `union` builder. This will ship the ImageSpec definition off to a hosted builder in your Union cluster. This unburdens your local machine from having to build and push an image yourself, resulting in faster iteration cycles. Moreover, the remote builder uses performance enhancements like layer caching and PyPI proxying to speed up builds even more.

In [ ]:
image = ImageSpec(
    name="boltz",
    packages=[
        "union",
        "flytekit==1.15",
        "union-runtime==0.1.11",
        "fastapi==0.115.11",
        "pydantic==2.10.6",
        "uvicorn==0.34.0",
        "python-multipart==0.0.20",
    ],
    apt_packages=["build-essential"],
    builder="union",
    commands=["pip install boltz==0.4.1"]
)

### Actor Workflow

Actors are a powerful primitve offering substantial performance improvements as well as unlocking certain capabilities not possible with regular tasks. By using a warm pod capable of accepting multiple task submissions, the overhead of pod scheduling and cleanup are removed. This results in faster iterations between tasks as well as enabling substantial improvements during large parallel executions.

We first define an ActorEnvironment using many of the same parameters we're accustomed to for regular tasks. Additionally, we define a replica count and a time-to-live to control parallelism capacity as well as how long to persist between task submissions. Once defined, the actor environment can be used in exactly the same way as the usual `@task` decorator.

The workflow itself requires no special treatment regarding actor tasks vs regular tasks. Finally, we call the workflow using `remote.execute`, pass in the input written out by the cell immediately below, and await a response. Once the workflow is submitted, head over to the console to watch the actor environment get provisioned and process the prediction!

Once the execution succeeds, the actor pod will remain active for the specified 10 minutes. Try changing something in the task itself and run the cell again. Everything will execute and return much faster.

In [ ]:
%%writefile prot_no_msa.yaml
version: 1
sequences:
  - protein:
      id: A
      sequence: QLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCG


In [ ]:
actor = ActorEnvironment(
    name="boltz-actor",
    replica_count=1,
    ttl_seconds=600,
    requests=Resources(
        cpu="2",
        mem="10Gi",
        gpu="1",
    ),
    container_image=image,
)

@actor.task
def simple_predict(input: FlyteFile) -> FlyteDirectory:
    input.download()
    out = "/tmp/boltz_out"
    os.makedirs(out, exist_ok=True)
    subprocess.run(["boltz", "predict", input.path, "--out_dir", out, "--use_msa_server"])
    return FlyteDirectory(path=out)

@workflow
def act_wf(input: FlyteFile) -> FlyteDirectory:
    return simple_predict(input=input)

execution = remote.execute(
    entity=act_wf, 
    inputs={"input": "prot_no_msa.yaml"}, 
    wait=True
)
output = execution.outputs
print(output)

### FastAPI App

Here, we initialize our FastAPI application, which will serve as the foundation for our API endpoints.

First, we implement some convenience functions: `package_outputs` and the asynchronous `generate_response`. The former is fairly self-explanatory, however the latter manages the execution of the Boltz process, yielding empty bytes during processing to maintain the connection. By implementing this as an asynchronous generator, we ensure our web server remains responsive during potentially long-running Boltz computations.

The heart of our implementation is the `/predict/` endpoint, which we define using FastAPI's decorator pattern. This endpoint accepts YAML input sequences and optional configuration parameters, optional MSA (Multiple Sequence Alignment) files, and additional CLI options.

Next, we construct and execute the Boltz command with appropriate parameters, including any custom options provided by the client. We've implemented flexibility here - if an MSA file is provided, we use it directly; otherwise, we instruct Boltz to use the `mmseqs2` MSA server for sequence alignments.
We're careful to implement robust error handling throughout our application, capturing and returning meaningful error messages if something goes wrong.

Finally, the results are streamed back to the client using FastAPI's StreamingResponse, which efficiently delivers the compressed output while setting appropriate headers to prompt the client to handle it as a downloadable file.

In [ ]:
%%writefile boltz_fastapi.py
import os
import io
import shutil
import asyncio
import tempfile
import traceback
import subprocess
from pathlib import Path
from typing import Optional, Dict, Any
from click.testing import CliRunner
from boltz.main import predict
from fastapi import FastAPI, File, UploadFile, Form, BackgroundTasks
from fastapi.responses import JSONResponse, StreamingResponse

app = FastAPI()

def package_outputs(output_dir: str) -> bytes:
    import io
    import tarfile

    tar_buffer = io.BytesIO()
    parent_dir = Path(output_dir).parent

    cur_dir = os.getcwd()
    with tarfile.open(fileobj=tar_buffer, mode="w:gz") as tar:
        os.chdir(parent_dir)
        try: 
            tar.add(Path(output_dir).name, arcname=Path(output_dir).name)
        finally: 
            os.chdir(cur_dir)

    return tar_buffer.getvalue()

async def generate_response(process, out_dir, yaml_path):
    try:
        while True:
            try:
                stdout, stderr = await asyncio.wait_for(process.communicate(), timeout=10.0)
                break
            except TimeoutError:
                yield b""  # Yield null character instead of spaces

        if process.returncode != 0:
            raise Exception(stderr.decode())

        print(stdout.decode())

        # Package the output directory
        tar_data = package_outputs(f"{out_dir}/boltz_results_{Path(yaml_path).with_suffix('').name}")
        yield tar_data

    except Exception as e:
        traceback.print_exc()
        yield JSONResponse(status_code=500, content={"error": str(e)}).body

@app.post("/predict/")
async def predict_endpoint(
    yaml_file: UploadFile = File(...),
    msa_file: Optional[UploadFile] = File(None),
    options: Optional[Dict[str, str]] = Form(None)
):
    yaml_path = f"/tmp/{yaml_file.filename}"
    with open(yaml_path, "wb") as buffer:
        shutil.copyfileobj(yaml_file.file, buffer)

    msa_path = f"/tmp/{msa_file.filename}"
    with open(msa_path, "wb") as buffer:
        shutil.copyfileobj(msa_file.file, buffer)

    # Create a temporary directory for the output
    with tempfile.TemporaryDirectory() as out_dir:
        # Call boltz.predict as a CLI tool
        try:
            print(f"Running predictions with options: {options} into directory: {out_dir}")
            # Convert options dictionary to key-value pairs
            options_list = [f"--{key}={value}" for key, value in (options or {}).items()]
            if msa_file and os.path.exists(msa_path):
                print(f"MSA file included at {msa_path}")
            else:
                options_list.append("--use_msa_server")
            command = ["boltz", "predict", yaml_path, "--out_dir", out_dir, "--cache", "/tmp/.boltz_cache"] + options_list
            print(f"Running command: {' '.join(command)}")
            process = await asyncio.create_subprocess_exec(
                *command,
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE
            )
            return StreamingResponse(generate_response(process, out_dir, yaml_path), media_type="application/gzip", headers={"Content-Disposition": f"attachment; filename=boltz_results.tar.gz"})

        except Exception as e:
            traceback.print_exc()
            return JSONResponse(status_code=500, content={"error": str(e)})


### Serving

Here we define our Union serving configuration, which specifies how our Boltz FastAPI service will be deployed and managed in your cluster.

First, we define our model artifact by creating a reference to the Boltz model that we materialized previously. This artifact definition includes important metadata such as the project, domain, name, and version, as well as partition information that describes the model's characteristics.

Following that, we define our FastAPI application deployment. The `App` class encapsulates all the specifications needed to run our service in a containerized environment:

- We give our application a name (`boltz-fastapi-notebook`) for identification within the Union system.
- We specify the same ImageSpec we've used throughout.
- We define an input via the Artifact's `query()` method, downloading it and mounting it at the specified path (`/tmp/.boltz_cache`).
- We establish resource limits for our application, including CPU, memory, GPU, and storage requirements.
- We set the port our application will listen on and specify the above file to include in the deployment.
- We define the command line arguments needed to start our FastAPI server using Uvicorn.
- We configure environment variables, such as enabling PyTorch MPS fallback for better compatibility.
- We set up auto-scaling parameters, including the minimum and maximum number of replicas, the scale-down timing, and the metric that triggers scaling (in this case, request rate).
- Finally, we specify the GPU accelerator type we need, which is an NVIDIA L40S in this implementation.

After configuring our application, we prepare for deployment by creating an `AppRemote` instance using the same `remote` object we've been using.

In the final step, we deploy our Boltz FastAPI application to the Union platform by calling the `deploy` method on our `AppRemote` instance. This initiates the deployment process, which will provision the necessary infrastructure, deploy our container, and make our Boltz service available according to the specifications we've defined. This will create a publicly accessible URL that leverages the same auth and RBAC as the rest of your cluster.

This deployment approach allows our Boltz service to automatically scale based on demand, efficiently utilize GPU resources when needed, and maintain high availability with minimum replicas always running.

In [ ]:
from datetime import timedelta

from union import Resources, ImageSpec
from union.app import App, ScalingMetric, Input
from flytekit.extras.accelerators import GPUAccelerator

boltz_model = Artifact(
    project="flytesnacks",
    domain="development",
    name="boltz-1",
    version="7c1d83b779e4c65ecc37dfdf0c6b2788076f31e1",
    partitions={
        "task": "auto",
        "model_type": "custom",
        "huggingface-source": "boltz-community/boltz-1",
        "format": "None",
        "architecture": "custom",
        "_u_type": "model",
    },
)

boltz_fastapi = App(
    name="boltz-fastapi-notebook",
    container_image=image,
    inputs=[
        Input(
            name="boltz_model", value=boltz_model.query(), download=True, mount="/tmp/.boltz_cache"
        ),
    ],
    limits=Resources(cpu="2", mem="10Gi", gpu="1", ephemeral_storage="50Gi"),
    port=8080,
    include=["./boltz_fastapi.py"],
    args=["uvicorn", "boltz_fastapi:app", "--host", "0.0.0.0", "--port", "8080"],
    env={
        "PYTORCH_ENABLE_MPS_FALLBACK": "1",
    },
    min_replicas=1,
    max_replicas=3,
    scaledown_after=timedelta(minutes=10),
    scaling_metric=ScalingMetric.RequestRate(1),
    accelerator=GPUAccelerator("nvidia-l40s"),
)

from union.remote._app_remote import AppRemote

app_remote = AppRemote(default_project="default", default_domain="development", union_remote=remote)

app_remote.deploy(boltz_fastapi)

### Trying It Out

Your app should now be fully provisioned and available at an automatically generated endpoint. Since we're using FastAPI, we get swagger docs for free. Head on over to that endpoint and add `/docs/` to the end to pull up the `/predict/` endpoint specification. You can then try it out by passing the files as named below. Once the prediction has run, you'll have the option to download a tarfile containing the results. Unarchiving it and looking in the predictions folder, you'll find a `.cif` file. This Crystallographic Information File contains the predicted structure of the below sequence. This can then be uploaded to [Molstar](https://molstar.org/viewer/) to view and interact with the structure.

## Final Thoughts

We've covered a lot in this compact example. We've captured the model itself in a convenient Artifact so that we can reference it across workflows and view important metadata. We ran a more traditional workflow in an accelerated way via Actors. We also stood up a persistent app for serving predictions in a flexible and cost effective way using Union Serving. 

All of this was accomplished programatically via UnionRemote from a Jupyter Notebook! There are many directions to go from here, however this represents an approachable and efficient way of prototyping fairly complex use cases.

In [ ]:
%%writefile prot.yaml
version: 1
sequences:
  - protein:
      id: A
      sequence: QLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCG
      msa: /tmp/seq.a3m

Writing prot.yaml


In [ ]:
%%writefile seq.a3m
>101
SDQLEDSEVEAVAKGLEEXYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAXISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>UniRef100_A0A8S0JH07	200	0.549	9.920E-56	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIAFAENELETNINNPY----DRVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGM-
>UniRef100_UPI00110EAFB6	199	0.977	2.558E-55	0	135	136	34	169	170
SDQLEDSEVEAVAKGLEEMYTNGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>UniRef100_UPI001CF7A1DD	185	0.539	1.616E-50	9	134	136	1	122	124
---------DDVANVLDQMFRNGVNEKNFTEYVYKNFSQKDIALAENKLETNINN----LYDRVPWDEMGGCIARKIREEFFAMTNVSLIVKYAQKKAWLELAKVVLRFVKANGLKTNTYIIAGQLAIWAVQCGL-
>UniRef100_A0A455TWS1	177	0.686	8.943E-48	1	134	136	35	168	170
-EQIKNSEVDTVAQGLEQMFSNGVSEENFKNYVNANFSSEEITKSEKELDVNLSNTSSPIQARVNWNGLGQCMANKIKDEFFAMINVGAIVAAAQKKAWKELAMTVLIFAKANGLKTNALIVAGQLAVWAVQCGL-
>UniRef100_UPI0012B3E02F	177	0.466	8.943E-48	4	134	136	44	176	179
----EQRQIDEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDEnaIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>UniRef100_A0A5Q2SBQ8	172	0.476	5.426E-46	9	134	136	1	128	131
---------DEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDEnaIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>UniRef100_A0A7U9RNI7	166	0.503	6.190E-44	5	134	136	43	181	185
-----DPEVAVVAQELEKIFANGVSQENLNKYVLKNFSNKELTVAEKELDVNYNpfslqskndNNSLHSVSVYGWNNLGQCMYNKIKDEFFEMVNIGVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>UniRef100_UPI00062A1226	162	0.286	1.456E-42	10	134	136	55	183	186
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>UniRef100_UPI0013568C51	162	0.445	2.738E-42	9	134	136	1	128	131
---------DKFAAVLEKMFAKGVTEENFKQYAQANYSEEELMIADREMNTNLSqiQDDDVIMYKMDWNALGSYMANKIKDELLAMISIGTIITYAKRKAWKELATIVIKYVAKAWVRTNVAFIAGQLAIWGLQCGI-
>UniRef100_UPI001FD87A89	159	0.533	1.821E-41	4	134	136	39	171	173
----EDKEIELVANELEAMFSNGVTENNLKNYVNENYDSRDISTAEKELNTSLkkTNYNYLLRSKFSWNKFGNCMVNEIKDEFFAMINVATIVKYAKKKSWKKLAGVVLKFAKANGLKTNVAIIAGQLAVWAIKCGI-
>UniRef100_UPI001A0EA69D	158	0.598	4.696E-41	3	134	136	44	173	175
---ISSQEVDQVAQALELMFDNNVSTSNFKKYVNNNFSDSEIAIAELELESRISN--SRSEFRVAWNEMGGCIAGKIRDEFFAMISVGTIVKYAQKKAWKELALVVLKFVKANGLKTNAIIVAGQLALWAVQCGL-
>UniRef100_UPI000DA36189	158	0.617	6.440E-41	7	134	136	48	173	175
-------EVDQVAQALELMFDNNVSTSNFKKYVNNNFSDSEIAIAELELESRISN--SRSEFRVAWNEMGGCIAGKIRDEFFAMISVGTIVKYAQKKAWKELALVVLKFVKANGLKTNAIIVAGQLALWAVQCGL-
>UniRef100_E1LSW0	157	0.518	1.211E-40	19	126	136	1	104	106
-------------------FRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPY----DRVPWDEMGGCIAGKIREDFFAMTNVSLIVKYAQKRAWLELAKVVLRFVKANRLKTNIYIIAGQLA---------
>UniRef100_A0A556UDB4	155	0.471	5.872E-40	0	134	136	34	173	177
SDSVQSQDVDTVATELQKMFAHGVSQENLNKYAKENFSKQELQAASRELDVNYLANtssttYSPFISMFSWNSMGKCMYNKIKDELFAMVNVGVIVKYAKKKAWKELAKVVIRVAKGAGVKTNAILVAGQLAVWAVACGM-
>UniRef100_A0A081PZB4	133	0.444	2.802E-32	4	93	136	75	160	161
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALSENELETNINNPY----DRVPWDEMGGCIAGKIRDEFFAMINVSLIVKYA------------------------------------------
>UniRef100_K0Z545	115	0.231	4.125E-26	37	132	136	144	251	263
-------------------------------------SSEQLALIVNSLNGAADNPYVDAQSDRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKSWKKVADLLKRYAKKEIAKKSGNIVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI000D25C0E4	108	0.213	1.654E-23	23	132	136	90	210	222
-----------------------VNEKLAAEYGFGR-SSEQLALIVNSLNGTSENPYVDAQSVRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKSWNKVAGLLKRYAKKEIANKGGNIVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C8F0653	107	0.493	3.107E-23	57	129	136	23	95	100
---------------------------------------------------------DVIMYKMDWNALGSYMANKIKDELLAMISIGTIITYAKRKAWKELATIVIKYVAKAWVRTNVAFIAGQLAIWG------
>UniRef100_UPI000D52D7A3	103	0.210	5.309E-22	17	132	136	87	210	222
-----------------KFFVD---EKLAAEYGFGR-SSEQLALIVNSLNGTSEHPHIDAQSVRSWGSFGSCVVTGVLGFSPFQIDYNLLGKYIYEKAWNKVAGLLKRYAKKEIAKKGGNIVLKQiikstpvgfaawLGVYAVGC---
>UniRef100_A0A133S201	103	0.386	7.277E-22	19	93	136	1	71	72
-------------------FRNGVNEKNFTECVYKNFSQKDIALAENKLETNINN----LYDRVPWDEMGGCIARKIREEFFAMTNVSLTVRYA------------------------------------------
>UniRef100_UPI000B18BA61	97	0.666	8.234E-20	72	134	136	0	62	66
------------------------------------------------------------------------MYNKIKDEFFAMVNIEVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVTAQLAVWAVQCGM-
>UniRef100_UPI00207479F9	93	0.244	1.924E-18	1	132	136	16	154	158
-EQIQDSqnQVDKVAKEFEDLFTNGiqINENSYainSDYLVQNYSSEEISGIVSLIEESSLINNTSTRSKRDIGSFLVCMKDKAVGDLKDMFNVGKFLVFIKTKAWKQAAEFAVKWLAKNGVKRNAVATAALLGWYGVQC---
>UniRef100_A0A2N8LAA9	93	0.259	1.924E-18	3	132	136	36	170	174
---ISQSEVDAVAIEFEKLFSNGIIisGNNYSinyDYLNNNYTSEEIQAFINLMASSELSPISSGRRKRSISSFVVCMKDKAVSDIADMFKVSAFVSFVQRKAWKEAAKFAVSWLAKNGIKRNVAATAALLSWYGIQC---
>UniRef100_UPI001FF59259	88	0.233	8.429E-17	5	132	136	30	162	166
-----QTEIESVASEFEQLFTKGIIisGNNYTfnhDYLTNNYTSDEIQAFIHLMDSTDLSPTFSKRRKRSIGSFAVCMKDKAVSDIADMFKVGAFVSFIQRKAWKEAAKFAVSWLAKNGIKRNVAATAALLSWYGIQC---
>UniRef100_UPI000407185C	87	0.745	2.168E-16	84	134	136	1	51	53
------------------------------------------------------------------------------------TNVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>UniRef100_UPI00135BC820	83	0.255	3.685E-15	5	132	136	38	170	174
-----QAEVDAVAVELENLFTngiliNGDSYTLNIDYLYKNYTPEEVDSFIALMKSSNLSSNTINRKKRSVNSFLVCMKDKAVADIADMFKVGAFVSFIQRKAWREAAQFAVSWLARNGIKRNVAATAALLSWYGVQC---
>UniRef100_A0A2N6UQ44	79	0.181	1.173E-13	3	132	136	47	200	206
---ITEAEIDAFSQDVADKFVilmrdgiiyDGSSFRSNDDVLSAAGFANEADLIEElvdSLNKGANNRYTSVYTDRSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIEKKAGSVVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C0EBA32	79	0.168	1.607E-13	3	132	136	48	201	213
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNEAdlIKELVDSLNNGANNRYANVYTERSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIKKKAGSVVLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI001C0F1009	78	0.162	4.127E-13	3	132	136	47	200	212
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNEAdlIKELVDSLNNGANNRYANVYTERSLSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEKAWNKVADLLQKYAKKEIKKKAGSVLLKQiikstpagfaawLGVYAVGC---
>UniRef100_UPI000214F415	66	0.346	2.716E-09	5	53	136	24	72	93
-----DPEVAIAAQELEKIFVDVVSQKNLNRYALKNFSNKELTVAEKELDVNYN----------------------------------------------------------------------------------
>UniRef100_UPI0012EE93E6	58	0.238	2.655E-06	62	124	136	108	170	202
--------------------------------------------------------------DRSWASFGECVVTGILGFSIFGIDYNLLGQYIKQKAWGAAAALLKKEAEKELKKQGAKIALKQ-----------
>UniRef100_UPI0011BE0601	57	0.141	6.774E-06	3	96	136	48	153	158
---ITEAEIDAFSQEVADKFVilmrdgiiyDGSSFRSNDDVLSAmGFSNgaDLIKELVDSLNNGANNRYANVYTERSWSSFGKCVVTGVLGFSPFQIDYNLLGKYIKEK---------------------------------------
>UniRef100_UPI000370F184	55	0.201	2.359E-05	29	132	136	98	206	211
-----------------------------KEVLISNLGYDEASATAESLGLTIDELVSDRTVpntngDRGVGEFLSCMKGHATDDLKSIFNVNAVSYYIGKEKYFEAALAAGKHLAKQGIRRNAYGLIGILAWYGVRC---
>UniRef100_A0A0G3H5S3	55	0.206	3.222E-05	4	132	136	51	180	185
----QEEQVEALA-ELLKEMDEAGSEAEVQQLFVERFGQDNLEDAAAQLGTNTSEVFreSDEVVEEGFGEFLQCIKGKATNDIKKALDVNVVAAFIGQKDYAKAAWAIVKHLAKQGIKRNAFAIAGMLAWWAWQC---
>UniRef100_UPI000F652D05	50	0.250	9.887E-04	63	134	136	111	182	185
---------------------------------------------------------------RDTGEFLSCMKGKASDDLKSIFDVNAIAVLIGQEKYGEAAVAAVKYLAKQGIKRNAAGIAGVLLFYAAKCSI-
>UniRef100_UPI00125CB70C	46	0.225	2.995E-02	53	132	136	97	176	181
-----------------------------------------------------NELTTDTTSFRGAPQFLSCMKSKVSNDLKSIFNINAIASLIGQQKYYEAAVKAVQYLAKQGIKRNVAGIAGALAFYGAKC---
>UniRef100_UPI0009761044	44	0.285	1.404E-01	67	132	136	108	174	176
-------------------------------------------------------------------SFWSCMKNQLLDmigyQTFQALLRGGIQGLIQRKAWKAAAKLLIRYL---GDGIGVGFIAAQLAWYAIRC---
>UniRef100_V9W2Y6	44	0.311	1.911E-01	72	132	136	0	56	60
------------------------------------------------------------------------MKDKLMQEFEELVNIGTLGVLIAEKKWTELAWTLIKK----GVKRNPWVLAGWLAWQAAKC---
>UniRef100_UPI001F1C6CF6	43	0.252	3.541E-01	70	132	136	53	115	120
----------------------------------------------------------------------SCMEGKVEDDVKEIFHVNAIAVLIGQEKYFEAAEEAVAYLAKQGVKRNAAAIASVLAFYGARC---
>UniRef100_UPI001F2B8987	43	0.252	3.541E-01	70	132	136	116	178	183
----------------------------------------------------------------------SCMEGKVEDDVKEIFHVNAIAVLIGQEKYFEAAEEAVAYLAKQGVKRNAAAIASVLAFYGARC---
>UniRef100_A0A2S0WEI1	42	0.228	4.819E-01	63	132	136	52	121	129
---------------------------------------------------------------RSVGEFLTCMRSEVTDDLKSIFDINAIAALIGREKYWEAAVEAVKFLAKQGIKRNVAGLAATLAFYGGKC---
>UniRef100_A0A9D1RZL8	39	0.238	5.627E+00	66	132	136	112	178	183
------------------------------------------------------------------GEFLSCIKGKVDAEVKAIFDVNAIAVLIGQENYFKAAKKAVDFLARQGIRKNAVGIAAVLAYHGARC---
>101
SDQLEDSEVEAVAKGLEEXYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAXISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>U6S4W9	158	0.985	3.319E-41	0	135	136	55	190	191
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>uvig_117295_3	158	0.985	3.319E-41	0	135	136	55	190	191
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARFNWNALGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>A0A1C0USB4	156	0.557	1.610E-40	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>MGYP000967978327	156	0.557	1.610E-40	4	134	136	45	171	173
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>R2N7C7	143	0.686	7.398E-36	1	134	136	35	168	170
-EQIKNSEVDTVAQGLEQMFSNGVSEENFKNYVNANFSSEEITKSEKELDVNLSNTSSPIQARVNWNGLGQCMANKIKDEFFAMINVGAIVAAAQKKAWKELAMTVLIFAKANGLKTNALIVAGQLAVWAVQCGL-
>A0A2A5L654	137	0.286	1.156E-33	10	134	136	55	183	186
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>3300014513.a:Ga0169742_10090_9	137	0.286	1.156E-33	10	134	136	55	183	187
----------AVAKELEKMFVDGDIENLNIDYLIAKYGKDEIQATERFIGISENEsrifPTEHRIVQRDLADIGNCMLGKLGEEIRSMVNVNTIVAYIDKKLWLEAAKAIVAKVAAQGIKRNAAVMATVLAWYAVQCGL-
>A0A256LDH3	134	0.503	1.053E-32	5	134	136	43	181	185
-----DPEVAVVAQELEKIFANGVSQENLNRYVLKNFSNKELTVAEKELDVNYN--PFSLQSKNDnnslhsvsvygWNNLGQCMYNKIKDEFFAMVNIGVIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>Q6DRR6	128	0.466	8.748E-31	4	134	136	44	176	179
----EQRQIDEVAAVLEKMFADGVTEENLKQYAQANYSEEELIIADNELNTNLSQIQDenAIMYKVDWGALGNCMANKIKDELLAMISVGTIIKYAQKKAWKELAKIVIKYVAKAGVKTNAALIAGQLAIWGLQCGI-
>MGYP000869820508	128	0.237	1.199E-30	4	133	136	55	192	196
----EEQQVEEVAKLLEKM-DKAPSREEAQRVLEDNFSQEELNGVAEELDIssdevltgaDAKDVGDAAATEEGFKDFYRCIKSKAGKDLRSALNVNAVMAAFGQKDYLKAAREIVKYLLKNGLKRNVFALAVTLGWYALQCS--
>MGYP000215453753	126	0.222	5.813E-30	4	132	136	51	184	188
----EKQQVEQLAKVLEKV-DNAPNREEAERIMVENFGEEGLEEAAKEIGVDPNNplgegsPTRSARGEVNEEGFVECMAGKVGDEIKGLLNINGVALALGQKDYPKMAAEIVKYLAKQGIKRNVGALALLLAWYGAQC---
>A0A0T8EFX5	116	0.504	2.127E-26	19	127	136	1	105	106
-------------------FRNGVNEKNFTEYVYKNFSQKDIALAENELETNINNPYD----RVPWDEMGGCIVGKIREDFFAMTNVSLIVKYAQKRAWLELAKVVLRFVKANRLKTNIYIIAGQLAF--------
>MGYP000013645973	116	0.244	2.127E-26	1	132	136	16	154	158
-EQIQDSqnQVDKVAKEFEDLFTNGiqINENSYainSDYLVQNYSSEEISGIVSLIEEISLINNTYTRSKRDIGSFLVCMKDKAVGDLKDMFNVGKFLVFIKTKAWKQAAEFAVKWLAKNGVKRNAVATAALLGWYGVQC---
>MGYP000875481199	113	0.232	1.936E-25	4	132	136	52	192	197
----EEQQVEEIAKFLEQV-DKAPSEKDAHRILEERFSQEELNGAAEELGVSPEEmfadggaqdgaklPEDPANNEESFGSFMRCIAGKAGNDLKSVIYSKDVLRAISKKQYKKAALAIVRHLAKQGIKRNIVVLGIQLGWYAFQC---
>MGYP000964187490	103	0.221	5.146E-22	5	103	136	30	133	134
-----QTEIESVASEFEQLFTKGIIisGNNYTfnhDYLTNNYTSDEIQAFIHLMDSTDLSPTFSKRRKRSIGSFAVCMKDKAVSDIADMFKVGAFVSFVQRKAWKEAAK--------------------------------
>A0A0K0G7I1	94	0.202	7.246E-19	0	132	136	10	172	176
SDDVEGAELAKLEQDLEFLFEEATTEEGDKYVLdeekaKDYFGEEnlpEIKILIKLINeeevteeefLNagiVPKHESYEEAKaevRDQNdnlvsnySWLGCMKTKIIAGTGLGFFGKGIDKLIEDKNWKKLSKEIIKIVGKNAVRGGAVGLAASLAVWSTMC---
>W7RBR4	91	0.250	1.235E-17	61	132	136	114	185	189
-------------------------------------------------------------SQFHTASWWGCLKEKIIDFTGLGFIGGGLEKMLKKKLWKKAATEIIKIVGKNAIRGGVLGLAGSLAWFSVRC---
>MGYP001084790427	85	0.971	1.389E-15	67	135	136	9	77	78
-------------------------------------------------------------------SLGSCVANKIKDEFFAMISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>MGYP001008199351	84	0.710	1.903E-15	66	134	136	15	83	85
------------------------------------------------------------------DEMGGCIAGKIRDEFFAMINVSLIVKYAQKKAWSELAKVVLRFVKANGLKTNIYIIAGQLAIWAVQCGL-
>MGYP001068281740	75	0.984	3.611E-12	0	63	136	34	97	115
SDQLEDSEVEAVAKGLEEMYANGVTEDNFKNYVKNNFAQQEISSVEEELNVNISDASTVVQARF------------------------------------------------------------------------
>MGYP000904613324	74	0.242	4.944E-12	67	132	136	14	79	83
-------------------------------------------------------------------SWTSCIKEKIMIATGIGFITGGMNKLIEEKAWKKLSLEIAKIVGKNAIKGGVVGLTASLAVWSIVC---
>MGYP000968663296	70	0.396	1.143E-10	4	56	136	64	116	117
----EQKQIDDVANVLEQMFRNGVNEKNFTEYVYKNFSQKDIALAENKLETNINNLY-------------------------------------------------------------------------------
>A0A133S201	70	0.378	2.141E-10	19	92	136	1	70	72
-------------------FRNGVNEKNFTECVYKNFSQKDIALAENKLETNINNLYD----RVPWDEMGGCIARKIREEFFAMTNVSLTVRY-------------------------------------------
>MGYP001149482190	69	0.678	4.009E-10	79	134	136	22	77	81
-------------------------------------------------------------------------------EFFAMVNIGLIVKYAKKKAWKELAKVVIRFAKGAGVRTNAAIVAAQLAVWAVQCGM-
>A0A0Q9UUB0	68	0.270	1.027E-09	38	132	136	83	175	203
--------------------------------------QQEIESSSCELaETTVINPIQPLAAKKSWKS---CMVASLKDHFGVALIevamTGGLWAYLQKKAYKEAAKLLIKI----GIGGNAIGLAATLTWYGTRC---
>A0A192YJB0	67	0.271	1.406E-09	67	132	136	117	183	185
-------------------------------------------------------------------SFWSCMKNQLLDmisyQTFQALLRGGIQGLIQRKAWKAAAKLLIRYLG-DGI--GVGFIAAQLSWYAIRC---
>MGYP000655452331	66	0.928	2.632E-09	80	135	136	7	62	63
--------------------------------------------------------------------------------IRDRISISAIVKAAQKKAWKELAVTVLRFAKANGLKTNAIIVAGQLALWAVQCGLS
>R8G7N7	62	0.238	8.256E-08	8	132	136	43	187	190
--------VEKLAEDLEFIMEKAAIRNsNDKVIdfdfdkLENRFGKlQEFKILKNEINndklnkheinqkqcTNISN-YNSLQAK-SWNGWKSCMVDALKDHFGVKIieiaFEGGLWGYLEKKAYKEAAKLLVKI----AVGSNVLGVTSFLVYYGAKC---
>UniRef100_A0A1X7GMK4	61	0.216	1.544E-07	7	132	136	38	178	206
-------QVDELAEDLEFLMEEAaiyDSENNVVgfhfDKLEDRFGeveelkmlQQEIESSSCELvetTTTETNSVQPLAAKKTWK---GCMIDSLKDHFGVAIIevamTGGLWAYLEQKAYKEAAKLLIKI----GVGGNVIGLAAFLTYYSAKC---
>A0A1X7GMK4	61	0.216	1.544E-07	7	132	136	38	178	206
-------QVDELAEDLEFLMEEAaiyDSENNVVgfhfDKLEDRFGeveelkmlQQEIESSSCELvetTTTETNSVQPLAAKKTWK---GCMIDSLKDHFGVAIIevamTGGLWAYLEQKAYKEAAKLLIKI----GVGGNVIGLAAFLTYYSAKC---
>MGYP001088613770	60	0.318	3.946E-07	5	70	136	29	92	98
-----DPEVAVAAQELEKIFVDAVSQKNLNRYALKNFSNKELTVAEKELDVNYN--PFSLQSKNDNNSLHS-----------------------------------------------------------------
>A0A1L6ZGN2	59	0.239	7.374E-07	66	132	136	107	173	182
------------------------------------------------------------------GTWGDCMIDSLKDHFGVAMIevalTGGLWSYLEKKAYKEAAKLLLKI----GIGGNVIGLVAFLTWYSAKC---
>A0A1J9VKJ1	55	0.252	1.674E-05	62	132	136	108	176	182
--------------------------------------------------------------KRS--AWTDCMVDAIKDHFGVAAVTaaleGGLWAYLEKKAYKEAAKLLVKF----AIGSNAVGLAGTLVYYGGKC---
>F7SG70	55	0.318	3.123E-05	5	70	136	24	87	93
-----DPEVAIAAQELEKIFVDVVSQKNLNRYALKNFSNKELTVAEKELDVNYN--PFSLQLKNDNNSLHS-----------------------------------------------------------------
>A0A0V8JI20	51	0.273	5.147E-04	64	132	136	107	175	181
----------------------------------------------------------------SNEAWKSCMIGAIKDHFGVAMVTaaleGGLWAYLEKKAYKEAAKLLVKF----AVGSNAVGLAGTLIYYGGVC---
>MGYP000101651737	46	0.236	2.903E-02	15	69	136	9	58	59
---------------LEQEFLISLI-KDKPETVYKNFSKKDIALAENKLEININNLYD----RVPWDEMG------------------------------------------------------------------
>A0A270AIV9	44	0.276	9.993E-02	86	132	136	1	43	71
--------------------------------------------------------------------------------------TGGLWANLEKKAYKEAAKLLVKI----GIGGNAIDLASFLTWYSARC---
>MGYP001000650989	44	0.276	1.853E-01	86	132	136	18	60	67
--------------------------------------------------------------------------------------TGGLWGYLKKKAYKEAAKLLVKI----GVGTNAATLAATLIYYGGKC---
>MGYP001053269802	44	0.216	1.853E-01	68	133	136	0	82	88
--------------------------------------------------------------------FGKCILNKMdlgeLKNLAKIIFKPATVRYLKSHAWKKASammvNAIVQYAPKKvaslAVKKFANLTLpgvgwASVAWWGAQCG--
>MGYP000578092484	42	0.315	8.644E-01	63	132	136	95	167	172
---------------------------------------------------------------RSPYDFGKCI---LKDYFGVYIDlvqgklWDSFVGYLQSEAWTEAAKIILKIIGKSASKANLIATAGQLALAAFNC---
